# Dead Reckoning, "by hand", in Java
This sheet intends to show the steps to go through to calculate Estimated Altitude (He) and Azimuth (Z) from scratch, to come up with a **Line of Positions**, without navigation tables.  
You might want to get some after that ;)  
> **_Note_**: To comply with the logarithm tables, we will use here **_decimal_** logathims.  
> natural ones would do to, but this is done for clarity, so you can compare the numbers when using your tables.


The equations to solve are:  
$$
He = arcsin((sin(L).sin(D)) + (cos(L).cos(D).cos(AHL)))
$$

$$
Z = arctan\left(\dfrac{sin(AHL)}{(cos(D).tan(D))-(sin(L).cos(AHL))}\right)
$$
The required precision implies the usage of logarithms.

In [44]:
import java.math.*;
import java.text.DecimalFormat;

In [45]:
static String decToSex(double v) {
    String s = "";
    double absVal = Math.abs(v);
    double intValue = Math.floor(absVal);
    double dec = absVal - intValue;
    int i = (int) intValue;
    dec *= 60D;
    DecimalFormat df = new DecimalFormat("00.00");
    return Integer.toString(i) + '\u00b0' + df.format(dec) + "'";
}

We start with the following values:  

Estimated position:  
$L = N 37^o 46'$  
$G = W 122^o 37'$

From the almanacs:  
$D = S 6^o 10.84'$  
$AHG = 135^o 03.5'$  
$AHL = 12^o 26.5'$

First we look for the Estimated Altitude He.
$$
He = arcsin((sin(L).sin(D)) + (cos(L).cos(D).cos(AHL)))
$$
We need to find the logarithms of:  
$ sin(L) = sin(37^o46') $  
$ sin(D) = sin(-6^o 10.84')$ rounded to $sin(-6^o 11')$  
$ cos(L) = cos(37^o 46') $. 
$ cos(D) = cos(-6^o 10.84')$ rounded to $cos(-6^o 11')$  
$ cos(AHL) = cos(12^o 26.5') $ rounded to $cos(12^o 26')$  

In [46]:
double lat = 37.0 + (46.0 / 60.0);
double lng = -(122.0 + (37.0 / 60.0));
double dec = -(6.0 + (11.0 / 60.0));
double ahl = 12.0 + (26.0 / 60.0);

### He, Direct Calculation

In [47]:
double directSinHe = (Math.sin(Math.toRadians(lat)) * Math.sin(Math.toRadians(dec))) + (Math.cos(Math.toRadians(lat)) * Math.cos(Math.toRadians(dec)) * Math.cos(Math.toRadians(ahl)));
double heDirect = Math.asin(directSinHe);
System.out.printf("He: %s, Bam!\n", decToSex(Math.toDegrees(heDirect)));

He: 44°32.92', Bam!


java.io.PrintStream@5df49f3e

### He, With Logarithms

Notice that we use the absolute values of the angles (for the logaritms to be happy).

In [48]:
double logSinLat = Math.log10(Math.sin(Math.toRadians(Math.abs(lat))));
double logSinDec = Math.log10(Math.sin(Math.toRadians(Math.abs(dec))));
double logCosLat = Math.log10(Math.cos(Math.toRadians(Math.abs(lat))));
double logCosDec = Math.log10(Math.cos(Math.toRadians(Math.abs(dec))));
double logCosAhl = Math.log10(Math.cos(Math.toRadians(Math.abs(ahl))));

In [49]:
System.out.printf("log(sin(L)) = %f\n", logSinLat);
System.out.printf("log(sin(D)) = %f\n", logSinDec);
System.out.printf("log(cos(L)) = %f\n", logCosLat);
System.out.printf("log(cos(D)) = %f\n", logCosDec);
System.out.printf("log(cos(AHL)) = %f\n", logCosAhl);

log(sin(L)) = -0.212931
log(sin(D)) = -0.967743
log(cos(L)) = -0.102092
log(cos(D)) = -0.002534
log(cos(AHL)) = -0.010307


java.io.PrintStream@5df49f3e

In [50]:
double logSinLatSinDec = logSinLat + logSinDec;
System.out.printf("log(sin(L).sin(D)) = (%f) + (%f) = %f\n", logSinLat, logSinDec, logSinLatSinDec);

log(sin(L).sin(D)) = (-0.212931) + (-0.967743) = -1.180675


java.io.PrintStream@5df49f3e

In [51]:
double sinLsinD = Math.pow(10, logSinLatSinDec);  // exponential base 10
System.out.printf("sin(L).sin(D) = %f\n", sinLsinD);

sin(L).sin(D) = 0.065967


java.io.PrintStream@5df49f3e

In [52]:
double logCosLCosDCosAHL = logCosLat + logCosDec + logCosAhl;
System.out.printf("log(cos(L).cos(D).cos(AHL)) = (%f) + (%f) + (%f) = %f\n", logCosLat, logCosDec, logCosAhl, logCosLCosDCosAHL);


log(cos(L).cos(D).cos(AHL)) = (-0.102092) + (-0.002534) + (-0.010307) = -0.114933


java.io.PrintStream@5df49f3e

In [53]:
double cosLcosDcosAHL = Math.pow(10, logCosLCosDCosAHL); // exponential base 10
System.out.printf("cos(L).cos(D).cos(AHL) = %f\n", cosLcosDcosAHL);

cos(L).cos(D).cos(AHL) = 0.767481


java.io.PrintStream@5df49f3e

In [54]:
// this sign evaluation may need some love...
int sign = (Math.abs(lat) == lat) ? 1 : -1;
sign *= (Math.abs(dec) == dec) ? 1 : -1;
sign *= (Math.abs(ahl) == ahl) ? 1 : -1;

double theSum = sinLsinD + (sign * cosLcosDcosAHL);  // SIGN !!!
System.out.printf("sin(L).sin(D) + cos(L).cos(D).cos(AHL) = %f\n", theSum);

System.out.printf("He: %s\n", decToSex(Math.toDegrees(Math.asin(theSum))));

sin(L).sin(D) + cos(L).cos(D).cos(AHL) = -0.701514
He: 44°32.92'


java.io.PrintStream@5df49f3e

### Z, direct

In [55]:
double tanZ = Math.sin(Math.toRadians(ahl)) / ((Math.cos(Math.toRadians(lat)) * Math.tan(Math.toRadians(dec))) - (Math.sin(Math.toRadians(lat)) * Math.cos(Math.toRadians(ahl))));
double zDirect = Math.abs(Math.atan(tanZ));
System.out.printf("Z: %s\n", decToSex(Math.toDegrees(zDirect)));

Z: 17°28.74'


java.io.PrintStream@5df49f3e

As L and D have different signs, we are behind the meridian time, Z goes from S to W:

In [56]:
System.out.printf("Z becomes %s\n", decToSex(180 + Math.toDegrees(zDirect)));

Z becomes 197°28.74'


java.io.PrintStream@5df49f3e

### Z, with logarithms

We need extra values

In [57]:
double logTanDec = Math.log10(Math.tan(Math.toRadians(Math.abs(dec))));
System.out.printf("log(tan(D)) = %f\n", logTanDec);

log(tan(D)) = -0.965209


java.io.PrintStream@5df49f3e

In [58]:
double logSinAHL = Math.log10(Math.sin(Math.toRadians(Math.abs(ahl))));
System.out.printf("log(sin(AHL)) = %f\n", logSinAHL);

log(sin(AHL)) = -0.666949


java.io.PrintStream@5df49f3e

In [59]:
double logCosLTanD = logCosLat + logTanDec;
System.out.printf("log(cos(L).tan(D)) = %f\n", logCosLTanD);

log(cos(L).tan(D)) = -1.067301


java.io.PrintStream@5df49f3e

In [60]:
double logSinLCosAHL = logSinLat + logCosAhl;
System.out.printf("log(sin(L).cos(AHL)) = (%f) + (%f) = %f\n", logSinLat, logCosAhl, logSinLCosAHL);

log(sin(L).cos(AHL)) = (-0.212931) + (-0.010307) = -0.223238


java.io.PrintStream@5df49f3e

In [61]:
double cosLtanD = Math.pow(10, logCosLTanD);
double sinLcosAHL = Math.pow(10, logSinLCosAHL);
double denom = - cosLtanD - sinLcosAHL;  // -1 * cosLtanD, different signes.
System.out.printf("Denominator: (%f) - (%f) = %f\n", -cosLtanD, sinLcosAHL, denom);

Denominator: (-0.085644) - (0.598084) = -0.683728


java.io.PrintStream@5df49f3e

In [62]:
double tanZ = Math.pow(10, logSinAHL) / denom;
System.out.printf("Z: %s\n", decToSex(Math.toDegrees(Math.atan(Math.abs(tanZ)))));

Z: 17°28.74'


java.io.PrintStream@5df49f3e

And like before:

In [63]:
System.out.printf("Z becomes %s\n", decToSex(180 + Math.toDegrees(Math.atan(Math.abs(tanZ)))));

Z becomes 197°28.74'


java.io.PrintStream@5df49f3e

---
This concludes the Java demo.